In [0]:
import pandas as pd

import os
import sys
sys.path.append('/Workspace/Users/angelarivera@bcp.com.pe/Modelo Applicant CEN/Model-ARCC-CP')  #aca esta el utils
from utils import *
from write_to_storage import *
from auxiliary_functions import *
from decorators import *
from visualization import *
#from model_functions import *
 

In [0]:
# #1. Train (entrenamiento)
# Es el conjunto de datos que se usa para ajustar los parámetros del modelo.
# El modelo aprende patrones, relaciones y estructuras a partir de estos datos.
# No se usa para evaluar el rendimiento final.

# # 2. Test (prueba)
# Se utiliza para evaluar el rendimiento del modelo después del entrenamiento.
# Ayuda a detectar si el modelo está sobreajustado (overfitting) o generaliza bien.
# No se usa durante el entrenamiento, solo después.

# # 3. Watch (monitoreo)
# Esta ventana se usa para monitorear el comportamiento del modelo en producción.
# Permite observar si el rendimiento se mantiene estable con datos recientes.
# No necesariamente se usa para ajustar el modelo, sino para detectar drift o cambios en la distribución de los datos.

# # 4. OOT (Out Of Time)
# Es un conjunto de datos completamente separado en el tiempo del resto.
# Se usa para simular cómo se comportaría el modelo en un escenario futuro.
# Es clave para validar la robustez temporal del modelo.


In [0]:
consulta = f"""
select codmes, CODCLAVEUNICOCLI, DEF12
from catalog_lhcl_prod_bcp.bcp_edv_fabseg.BD_CLIENTE_PD_BHV_TRONCAL_PTE7 a
where codmes between 202306  and 202408
and MONTO_TOTAL_Activ_Tot > 0 
and FLG_CLIENTE_PERFIL_MALO = 0
"""

base = spark.sql(consulta)
base.createOrReplaceGlobalTempView('base')
 
df = base.toPandas()
 
df.head(2)
 

,codmes,CODCLAVEUNICOCLI,DEF12
0,202401,0003e204df5162b614c65e243045e609ba602808cfbbeb...,0
1,202308,00058195f552e8828a589a8e85f6c7822d12e8815389e3...,0


In [0]:
df = df.sort_values(by=['codmes', 'CODCLAVEUNICOCLI'], ascending=[True, True]) #1094758

In [0]:
df2 = df[['CODCLAVEUNICOCLI']].drop_duplicates()
df2.head()

,CODCLAVEUNICOCLI
772040,000013983159f9ab9e95ea160352ade6141a5b35d81b88...
520567,0001cff0d2d0d81d3fbd0c65d3f749b699d4767f150935...
848420,0003d6396bb06f7dabc8dd8809c4f66891d59b0b63c6c3...
788183,0003e204df5162b614c65e243045e609ba602808cfbbeb...
340670,00045804a51c3cc2681525676b87219b1bfcbb9474045c...


In [0]:
from sklearn.model_selection import train_test_split
train_data,oot_data=train_test_split(df2,test_size=0.50,random_state=123)

In [0]:
clientes_seleccionados_global = set()
clientes_seleccionados_global.update(train_data['CODCLAVEUNICOCLI'])
 
fec_oot = [202406,202407,202408]
clientes_periodo_train = df[~df.codmes.isin(fec_oot)][df[~df.codmes.isin(fec_oot)]['CODCLAVEUNICOCLI'].apply(lambda x: x in clientes_seleccionados_global)]
clientes_periodo_train.head()

,codmes,CODCLAVEUNICOCLI,DEF12
772040,202306,000013983159f9ab9e95ea160352ade6141a5b35d81b88...,0
788183,202306,0003e204df5162b614c65e243045e609ba602808cfbbeb...,0
340670,202306,00045804a51c3cc2681525676b87219b1bfcbb9474045c...,0
1072767,202306,0006f822757faf090bde52f179e4ce7c58bba6fe47156c...,0
367820,202306,00072fbad50705a241f35a7ed154817b0ca4c2eead278c...,0


In [0]:
clientes_seleccionados_global = set()
clientes_seleccionados_global.update(oot_data['CODCLAVEUNICOCLI'])

In [0]:
clientes_periodo_oot = df[df.codmes.isin(fec_oot)][df[df.codmes.isin(fec_oot)]['CODCLAVEUNICOCLI'].apply(lambda x: x in clientes_seleccionados_global)]
clientes_periodo_oot.head()

,codmes,CODCLAVEUNICOCLI,DEF12
335215,202406,000200c460608bb27c0e21d39db24028b878ef80de6d03...,0
974350,202406,0002cb47bae0a03d2142064ca1c81a6c985ff13d80d3f0...,0
263914,202406,0002e1ee722982472159abda2bc77745bfcf4b61e0c5b1...,0
104855,202406,0004a87edfb5af73593c46e7f4e93c70312d2d49ab5581...,0
772042,202406,0004b3ad8490e55fbcf2846ecefc16a33c59f6af9a80f2...,0


In [0]:
pd.merge(clientes_periodo_train,clientes_periodo_oot,on=['CODCLAVEUNICOCLI'],how='inner')

,codmes_x,CODCLAVEUNICOCLI,DEF12_x,codmes_y,DEF12_y


In [0]:
base = pd.concat([clientes_periodo_train,clientes_periodo_oot],axis=0,ignore_index=True)
base.head()

,codmes,CODCLAVEUNICOCLI,DEF12
0,202306,000013983159f9ab9e95ea160352ade6141a5b35d81b88...,0
1,202306,0003e204df5162b614c65e243045e609ba602808cfbbeb...,0
2,202306,00045804a51c3cc2681525676b87219b1bfcbb9474045c...,0
3,202306,0006f822757faf090bde52f179e4ce7c58bba6fe47156c...,0
4,202306,00072fbad50705a241f35a7ed154817b0ca4c2eead278c...,0


In [0]:
pd.crosstab(df.codmes,df.DEF12,normalize='index')*100

DEF12,0,1
codmes,,
202306,87.152235,12.847765
202307,86.991834,13.008166
202308,86.825715,13.174285
202309,86.951626,13.048374
202310,87.184463,12.815537
202311,87.581571,12.418429
202312,88.019300,11.980700
202401,88.588609,11.411391
202402,89.218204,10.781796


In [0]:
pd.crosstab(base.codmes,base.DEF12,normalize='index')*100

DEF12,0,1
codmes,,
202306,87.312951,12.687049
202307,87.094290,12.905710
202308,86.954939,13.045061
202309,87.075050,12.924950
202310,87.270094,12.729906
202311,87.577727,12.422273
202312,88.100604,11.899396
202401,88.617909,11.382091
202402,89.308450,10.691550


In [0]:
pd.crosstab(df.codmes,df.DEF12)


DEF12,0,1
codmes,,
202306,59613,8788
202307,59017,8825
202308,57575,8736
202309,59461,8923
202310,60649,8915
202311,62542,8868
202312,63667,8666
202401,64380,8293
202402,65653,7934


In [0]:
pd.crosstab(base.codmes,base.DEF12) 

DEF12,0,1
codmes,,
202306,29758,4324
202307,29410,4358
202308,28656,4299
202309,29609,4395
202310,30130,4395
202311,31126,4415
202312,31666,4277
202401,32085,4121
202402,32828,3930


In [0]:
base.to_csv("/Workspace/Users/sherlytsalazar@bcp.com.pe/02_Proyectos/06_Construccion_PD_Troncal_BHV/01_Naive/base_muestreo.csv", index=False)

In [0]:
base = base.sort_values(by=['codmes', 'CODCLAVEUNICOCLI'], ascending=[True, True]) #1094758

In [0]:
data_base = base[~base.codmes.isin(fec_oot)]
data_oot = base[base.codmes.isin(fec_oot)]
print(data_base.shape)
print(base.shape)


## 
data_train, data_test = train_test_split(data_base,
                                         stratify = data_base[['DEF12','codmes']],
                                         train_size = 0.6,
                                         shuffle = True,
                                         random_state = 123)
 
 ## aca esta el 40% restante: 20% watch, 20% oot

data_test, data_watch = train_test_split(data_test,
                                         stratify = data_test[['DEF12','codmes']],
                                         train_size = 0.5,
                                         shuffle = True,
                                         random_state = 123)
 
# Concatenate subsets
development_sample = pd.concat([data_train.assign(sample = 'train'),
                                data_watch.assign(sample = 'watch'),
                                data_test.assign(sample = 'test'),
                                data_oot.assign(sample = 'oot')], axis = 0, ignore_index = True)
                               
print(development_sample.shape)

(426431, 3)
(545761, 3)
(545761, 4)


In [0]:
df1 = development_sample[['codmes','CODCLAVEUNICOCLI','DEF12','sample']]
df1.head()

,codmes,CODCLAVEUNICOCLI,DEF12,sample
0,202401,4bc3e2c716ffd082b52c3668aab38a17c93a89f903c4bb...,0,train
1,202403,cc2eb1e9227390e3178aaec3f511fffe6b50237024ef36...,0,train
2,202401,9507c7e3b60e83ebb4860047ddf7f4b91d046ec0bfe8ce...,0,train
3,202307,51436e90abbc21be64b87875ecdb6588d267a70e9f9706...,1,train
4,202306,d105181c0fa9cefe4406ea3085868f7e2888f1729ddab5...,0,train


In [0]:
df2 = safe_pandas_to_spark(df1)

In [0]:
## Muestreo
write_to_storage_partitioned(df = df2,
                                 tableName = ('SAMPLE_BEHAVIOR').upper(),
                                 partitionField = 'CODMES',
                                 dropOld = True,
                                 squad = 'fabrica',
                                 location = None,
                                 npart = 20)

DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.SAMPLE_BEHAVIOR

      CREATE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.SAMPLE_BEHAVIOR
      USING DELTA
      PARTITIONED BY (CODMES)
      LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/prod/bcp/edv/mmgr/FABRICA/SAMPLE_BEHAVIOR'
      AS SELECT * FROM SAMPLE_BEHAVIOR
    
2025-09-23 16:20:55 | RSS: 916.34 MB | VMS:  8.68 GB | ELAPSED: 00:00:30.236


In [0]:
%sql
select * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.SAMPLE_BEHAVIOR

codmes,CODCLAVEUNICOCLI,DEF12,sample
202311,8d5660d49d2aa019c3a31bcc93f3131a9cbe599b01c9e77fa2cdc0f30bc1bd480a7a3fa681e6cf2bf5f78646501a12ef99c531d1926a6693ac07ea46d638c803,0,test
202311,4bb875dbc900e443a00554fa803b771bca2e4aea8a910f8466cfa617850c6972be9d44a5cca31ad92e606cc703ed4fcab3a7aff9bb8ae2588fd97b1c9c4be6e2,1,watch
202311,af6ef7896b47e7fd3033ff0b12daa2e699131f7d1d77348dda11575f34318868fbb6da5aef49fb84f94eb8d5d3a0538f7600289858d9906bd5b271f503e63c29,0,train
202311,f75b9a6932624448108a786081f2741acf0b4b85ed175a07a299abf5ad4637e0415ef567042f6981607c9e56db1e8c94c4fa3572da2bcadf0867cf0c72a2c532,0,train
202311,71382556dbdaa3405bc1a11594a93d1669a9dc1b16fd9607a2bd5e4432b69b246a57ae5c871acd4d7b2b552c5e99b9962573b68b3c4873abf73efec4d8015eba,0,train
202311,82a4c61f2cd08b1a87a6ad760dbc85daa33f9d7360c3a850534305cf24954c544527e9a22005705f94be65032bc90e85c034dcb1bf18cd363740685c7afa7a20,0,train
202311,cf117a34767304e6bd64f90156d18ab716df11b03bdb12fb49b82bbcab4d29f9ad90997486d703c0e74fe87c001b8311e4db2574e677c58c490d2a2ef421215c,0,train
202311,ef1e47a6a3e2f170c3c8747a56b55fb6cf012c4fa5f93f0abb3668e33d3b8a074a5071fb4ea8c331250bdffd77e96346fe752e74c133cf9633b00d9429e270a5,1,test
202311,cdff753f4a06262d7ee15ec682c7870f6a78e00724763872ea9e24ec5091a404525ece379db4656e435e6596672b06247e72e2aaac418a7ba011152c866bd8bb,0,test
202311,54eb90176cf195965b5361f9f9295a924de1e1b592bb9b116f8ffb4e3b2f6962f409849dbe82a8361e9039fad7f6487e2f607256c6ffb021d351c96237e79f86,0,train
